In [1]:
from context import omphalos

import numpy as np
import matplotlib as pyplot
# Import Omphalos modules.
from omphalos import generate_inputs as gi
from omphalos import file_methods as fm
from omphalos import my_metrics as mm
from omphalos import omphalos_plotter as op
from omphalos import attributes as attr
from omphalos import labels as lbls
from omphalos import spatial_constructor

import sys
import subprocess
import pickle

import warnings
import numpy as np
import gpflow
import tensorflow as tf

from matplotlib import pyplot as plt

from gpflow.ci_utils import ci_niter, ci_range
from gpflow.models import VGP, GPR, SGPR, SVGP
from gpflow.optimizers import NaturalGradient
from gpflow.optimizers.natgrad import XiSqrtMeanVar
from gpflow import set_trainable

%matplotlib inline
%precision 4

np.random.seed(0)
tf.random.set_seed(0)

N, D = 100, 2
batch_size = 50

# inducing points
M = 10

inducing_variable = tf.random.uniform((M, D))
adam_learning_rate = 0.01
iterations = ci_niter(5)
autotune = tf.data.experimental.AUTOTUNE

In [2]:
train_set = fm.unpickle('data/1086_data.pickle')

attributes_df = attr.initial_conditions(train_set, primary_species=True)
labels_df = lbls.secondary_precip(train_set)

/usr/local/lib/python3.8/site-packages/pandas/core/frame.py:6390: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = expressions.where(mask, this, that)


In [3]:
x = attributes_df.loc[:, ['SO4--','Ca++', 'Formaldehyde', 'CO2(aq)']].xs(1, level=1)
y = labels_df.xs(9, level=1)['Calcite'] + labels_df.xs(9, level=1)['Calcite44']


x = x.to_numpy() 
y = y.to_numpy().reshape(-1,1)
y = y * 1e4
x = np.log(x)

# Fit and optimise

In [4]:
%%time

# Genereate varational Gaussian process
vgp = VGP((x,y), kernel=gpflow.kernels.RBF(lengthscales=[1,1,1,1], variance=1, active_dims=[0, 1, 2, 3]), likelihood=gpflow.likelihoods.Gaussian())
vgp.elbo().numpy()

# Natural gradients optimisation
natgrad_opt = NaturalGradient(gamma=1.0)
variational_params = [(vgp.q_mu, vgp.q_sqrt)]
natgrad_opt.minimize(vgp.training_loss, var_list=variational_params)
vgp.elbo().numpy()

from gpflow.utilities import print_summary
print_summary(vgp)

╒═════════════════════════╤═══════════╤══════════════════╤═════════╤═════════════╤═════════════════╤═════════╤══════════════════════════════════════════╕
│ name                    │ class     │ transform        │ prior   │ trainable   │ shape           │ dtype   │ value                                    │
╞═════════════════════════╪═══════════╪══════════════════╪═════════╪═════════════╪═════════════════╪═════════╪══════════════════════════════════════════╡
│ VGP.kernel.variance     │ Parameter │ Softplus         │         │ True        │ ()              │ float64 │ 1.0                                      │
├─────────────────────────┼───────────┼──────────────────┼─────────┼─────────────┼─────────────────┼─────────┼──────────────────────────────────────────┤
│ VGP.kernel.lengthscales │ Parameter │ Softplus         │         │ True        │ (4,)            │ float64 │ [1., 1., 1....                           │
├─────────────────────────┼───────────┼──────────────────┼─────────┼────────

## Draw a sample from posterior

In [12]:
%%time

plot_var = 1

# We define a function that plots the model's prediction (in the form of samples) together with the data.
# Importantly, this function has no other argument than `fig: matplotlib.figure.Figure` and `ax: matplotlib.figure.Axes`.

# plot the function posterior
samp = 50
xx = np.ones((samp, 4))
xx[:,0] = 26.79609e-03
xx[:,1] = 1.01838314236E-02
xx[:,2] = 0.55e-01
xx[:,3] = 2.00e-03
xx[:,plot_var] = np.linspace(1e-15, 0.2, samp)

xx = np.log(xx)

mean, var = vgp.predict_f(xx)

CPU times: user 26.2 s, sys: 15.5 s, total: 41.7 s
Wall time: 28.4 s


In [24]:
sweep[2].condition_blocks['Seawater'].minerals

{'FeS(am)': ['0.00001', '0.2'],
 'FeS34(am)': ['4.5090014600E-07', '0.2'],
 'SiO2(am)': ['0.00', '0.2'],
 'Quartz': ['1E-10', '1.0'],
 'Siderite': ['1E-10', '1.0'],
 'Calcite': ['0.31335679', '1.0'],
 'Calcite44': ['0.00664321', '1.0'],
 'K-Feldspar': ['0.02', 'ssa', '55.0'],
 'Kaolinite': ['0.02', 'ssa', '55.0'],
 'C5H7O2NSO4(s)': ['5.189E-6']}

In [18]:
sweep = fm.unpickle('data/1086_ca_sweep.pickle')

sweep_attr = attr.initial_conditions(sweep, primary_species=True, mineral_vols=True)
sweep_lbls = lbls.secondary_precip(sweep)

sweep_x = sweep_attr.loc[:, 'Ca++'].xs(1, level=1)
sweep_y = sweep_lbls.xs(9, level=1)['Calcite'] + sweep_lbls.xs(9, level=1)['Calcite44']
#so4_y = so4_labels.sum(level=0)['Calcite'] +  so4_labels.sum(level=0)['Calcite44']

/usr/local/lib/python3.8/site-packages/pandas/core/frame.py:6390: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[col] = expressions.where(mask, this, that)


## Plot sample

In [19]:
plt.style.use('seaborn-talk')


def plot_prediction(fig, ax):
    #Ypred = vgp.predict_f_samples(Xnew, full_cov=True, num_samples=20)
    #ax.plot(np.exp(Xnew[:,1]), np.squeeze(Ypred).T, "C1", alpha=0.2)
    ax.plot(np.exp(xx[:,plot_var]) * 1000, mean * 1e-4, label='GPR Fit')
    ax.fill_between(np.exp(xx[:, plot_var]) * 1000, (mean[:,0] - 2 * np.sqrt(var[:,0])) * 1e-4, (mean[:,0] + 2 * np.sqrt(var[:,0]))*1e-4, color="C0", alpha=0.2, label='95% Confidence Interval')
    ax.scatter(sweep_x * 1000, sweep_y, c='k', marker='+',label='RTM Result')
    #ax.set_xlim(0, 0.1)


x_labels =  ['[SO$_4^{2-}$] (mM)','[Ca$^{2+}$] (mM)', '[Formaldehyde] (mM)', '[CO$_{2(aq)}$] (mM)']
    
# Let's check if the function does the desired plotting
fig = plt.figure(figsize=(16,9))
ax = fig.subplots()
plot_prediction(fig, ax)
ax.set_xlabel(x_labels[plot_var], fontsize=20)
ax.set_ylabel('Net calcite precipitation (vol. frac.)', fontsize=20)
ax.legend(fontsize=20)
ax.tick_params('both', labelsize=18)
ax.set_xlim(0, 50)
ax.set_ylim(-0.00025, 0.00150)

path='/Users/angus/PhD_pres/fit_plots/{}.jpg'.format(plot_var)
fig.savefig(path, dpi=300)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## 2D sampling

In [132]:
%%time


num = 20
upper = 0.05
lower = 1e-5
x1 = np.linspace(lower, upper, num)
x2 = np.linspace(lower, upper, num)
x1_mesh,x2_mesh = np.meshgrid(x1, x2)
X = np.dstack([x1_mesh, x2_mesh]).reshape(-1, 2)

control_vals = np.zeros((num**2, 2))
control_vals[:, 0] = 0.55e-1 # Formaldehyde
control_vals[:, 1] = 2.00e-3 # CO2(aq)

X = np.hstack((X, control_vals))

vals, vars = vgp.predict_f(np.log(X))

CPU times: user 34 s, sys: 15.7 s, total: 49.8 s
Wall time: 34.2 s


In [25]:
%%time

#Finding best parameter set!
num = 8
upper = 0.05
lower = 1e-5
x1 = np.linspace(lower, upper, num)
x2 = np.linspace(lower, upper, num)
x3 = np.linspace(lower, upper, num)
x4 = np.linspace(lower, upper, num)

X = np.stack(np.meshgrid(x1, x2, x3, x4), -1).reshape(-1, 4)

vals, var = vgp.predict_f(np.log(X))

max_index = np.argmax(vals)

print(max_index)
print(X[max_index])

vals[max_index] * 1e-4

3903
[0.0286 0.05   0.05   0.05  ]
CPU times: user 2min 4s, sys: 24.6 s, total: 2min 28s
Wall time: 1min 8s


<tf.Tensor: shape=(1,), dtype=float64, numpy=array([0.0014])>

In [5]:
%%time


num = 20
upper = 0.05
lower = 1e-5
x1 = np.linspace(lower, upper, num)
x2 = np.linspace(lower, upper, num)
x1_mesh,x2_mesh = np.meshgrid(x1, x2)
X2 = np.dstack([x1_mesh, x2_mesh]).reshape(-1, 2)
control_vals = np.zeros((num**2, 2))

control_vals[:, 0] = 1.01838314236E-02 # Ca++
control_vals[:, 1] = 2.00e-3 # CO2(aq)


XX = np.hstack((X2[:,0].reshape(-1,1), control_vals[:,0].reshape(-1,1), X2[:,1].reshape(-1,1), control_vals[:,1].reshape(-1,1)))
vals, var = vgp.predict_f(np.log(XX))

CPU times: user 34.3 s, sys: 15.4 s, total: 49.7 s
Wall time: 33.6 s


In [7]:
%matplotlib widget
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

plt.style.use('seaborn-talk')


axis_labels =  ['[SO$_4^{2-}$] (mM)','[Ca$^{2+}$] (mM)', '[Formaldehyde] (mM)', '[CO$_{2(aq)}$] (mM)']


fig = plt.figure(figsize=(20,20))
ax = fig.add_subplot(111, projection='3d')
ax.set_xlabel(axis_labels[0], fontsize=20, labelpad=20)
ax.set_ylabel(axis_labels[2], fontsize=20, labelpad=20)
ax.set_zlabel('Net calcite precipitation (vol. frac.)', fontsize=20, labelpad=30)
ax.tick_params('both', labelsize=18)
ax.ticklabel_format(axis='z', style='sci', scilimits=(0,0))
ax.set_zlim(-0.5e-4, 14e-4)
ax.view_init(elev=15, azim=-128)

def shape(obj):
    print(np.shape(obj))
    

ax.plot_trisurf(XX[:,0]*1000, XX[:,2]*1000, vals.numpy().reshape(-1) * 1e-4,  cmap=plt.cm.magma)



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
fig.savefig('/Users/angus/PhD_pres/3D_plots/1086_form_so4', dpi=300)